In [1]:
pip install transformers

     |████████████████████████████████| 890kB 8.1MB/s 
     |████████████████████████████████| 3.0MB 26.9MB/s 
     |████████████████████████████████| 1.1MB 51.9MB/s 
     |████████████████████████████████| 890kB 56.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=7811d1abccba4768306ec9919c4b758d271fec4d39735642443f695373739442
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split

In [26]:
data = pd.read_csv('/content/hindi_tr.csv')
data2 = pd.read_csv('/content/final.csv')

data = data[['text', 'task2']]
data2 = data2[['text', 'task_2']]
data2 = data2.rename(columns={'task_2': 'task2'})
df_row = pd.concat([data, data2])

data=df_row

data.head()

,text,task2
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,HATE
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",NONE
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,HATE
3,Yey nina khothani labafazi benu phambili Finis...,PRFN
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,HATE


In [27]:
data['task2'] = pd.Categorical(data['task2'])
data['task2'] = data['task2'].cat.codes
data.head()

,text,task2
0,1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...,0
1,"RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...",2
2,RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...,0
3,Yey nina khothani labafazi benu phambili Finis...,5
4,RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...,0


In [28]:
data, data_test = train_test_split(data, test_size = 0.2)
data.shape,data_test.shape

((6102, 2), (1526, 2))

In [29]:
model_name = 'bert-base-uncased'
max_length = 64
config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [30]:
bert = transformer_model.layers[0]
input_ids = Input(shape=(max_length,), name='input_ids', dtype='int32')
inputs = {'input_ids': input_ids}
bert_model = bert(inputs)[1]
dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = dropout(bert_model, training=False)
task2 = Dense(units=len(data.task2.value_counts()), kernel_initializer=TruncatedNormal(stddev=config.initializer_range), name='task2')(pooled_output)
outputs = {'task2': task2}
model = Model(inputs=inputs, outputs=outputs, name='BERT_MultiLabel_MultiClass')
model.summary()


Model: "BERT_MultiLabel_MultiClass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 64)]              0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 64, 768), (None,  109482240 
_________________________________________________________________
pooled_output (Dropout)      (None, 768)               0         
_________________________________________________________________
task2 (Dense)                (None, 6)                 4614      
Total params: 109,486,854
Trainable params: 109,486,854
Non-trainable params: 0
_________________________________________________________________


In [31]:
optimizer = Adam(
    learning_rate=5e-05,
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
loss = {'task2': CategoricalCrossentropy(from_logits = True)}
metric = {'task2': CategoricalAccuracy('accuracy')}

In [32]:
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)
y_issue = to_categorical(data['task2'])

In [33]:
x = tokenizer(
    text=data['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
history = model.fit(
    x={'input_ids': x['input_ids']},
    y={'task2': y_issue},
    validation_split=0.2,
    batch_size=64,
    epochs=10)

Epoch 1/10
77/77 [==============================] - 65s 848ms/step - loss: 1.1527 - accuracy: 0.5757 - val_loss: 1.1144 - val_accuracy: 0.5815
Epoch 2/10
77/77 [==============================] - 66s 857ms/step - loss: 1.0437 - accuracy: 0.6140 - val_loss: 1.0230 - val_accuracy: 0.6265
Epoch 3/10
77/77 [==============================] - 66s 857ms/step - loss: 0.9384 - accuracy: 0.6646 - val_loss: 0.9327 - val_accuracy: 0.6634
Epoch 4/10
77/77 [==============================] - 66s 858ms/step - loss: 0.8417 - accuracy: 0.6949 - val_loss: 0.9232 - val_accuracy: 0.6658
Epoch 5/10
77/77 [==============================] - 66s 860ms/step - loss: 0.7701 - accuracy: 0.7119 - val_loss: 0.9226 - val_accuracy: 0.6880
Epoch 6/10
77/77 [==============================] - 66s 860ms/step - loss: 0.6984 - accuracy: 0.7376 - val_loss: 0.9352 - val_accuracy: 0.6855
Epoch 7/10
77/77 [==============================] - 66s 860ms/step - loss: 0.6367 - accuracy: 0.7617 - val_loss: 1.0075 - val_accuracy: 0.6888

In [34]:
test_y_issue = to_categorical(data_test['task2'])
test_x = tokenizer(
    text=data_test['text'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = False,
    verbose = True)
model_eval = model.evaluate(
    x={'input_ids': test_x['input_ids']},
    y={'task2': test_y_issue}
)

48/48 [==============================] - 7s 139ms/step - loss: 1.1340 - accuracy: 0.6409


In [35]:
test_x

{'input_ids': <tf.Tensor: shape=(1526, 64), dtype=int32, numpy=
array([[  101, 19387,  1030, ..., 29879,  1331,   102],
       [  101,  1325, 29877, ...,  9333,  1012,   102],
       [  101,  1001,  4911, ...,   102,     0,     0],
       ...,
       [  101,  1335, 29879, ...,   999,  1030,   102],
       [  101,  1338, 29874, ..., 29859, 29868,   102],
       [  101, 19387,  1030, ..., 29863,  1330,   102]], dtype=int32)>}

In [36]:
preds = model.predict(test_x['input_ids'])

In [37]:
preds=preds['task2'].argmax(axis=-1)

In [38]:
preds

array([2, 2, 2, ..., 2, 2, 2])

In [39]:
test_y_issue=test_y_issue.argmax(axis=-1)

In [40]:
test_y_issue

array([2, 2, 2, ..., 2, 2, 0])

In [41]:
from sklearn.metrics import classification_report
print(classification_report(test_y_issue,preds))

              precision    recall  f1-score   support

           0       0.15      0.11      0.12       148
           2       0.75      0.83      0.79       880
           4       0.28      0.24      0.26       217
           5       0.70      0.65      0.67       281

    accuracy                           0.64      1526
   macro avg       0.47      0.45      0.46      1526
weighted avg       0.61      0.64      0.63      1526

